# ChatBot RAG com documentos PDF
---
Este notebook conecta LangChain com API Groq e faz as primeiras interações.

### IMPORTS

In [46]:
import os
import time
from dotenv import load_dotenv
from typing import Dict, Any, List

## Groq
from langchain_groq import ChatGroq

## core
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage

### CONFIGURAÇÃO DA API

In [6]:
load_dotenv()

GROQ_API_KEY = os.environ["GROQ_API_KEY"]

if not GROQ_API_KEY:
    raise ValueError("GROQ_API_KEY não foi configurada!")

print("API Key configurada!")

API Key configurada!


### CONFIGURAÇÕES GLOBAIS

In [70]:
CONFIGS = {
    "model":"llama-3.1-8b-instant",
    "temperature": 0.1,
    "max_tokens": 2048,
    "max_messages":10
}

print(f"""
CONFIGURAÇÕES
    - Modelo LLM              - {CONFIGS['model']}
    - Temperatura             - {CONFIGS['temperature']}
    - Max tokens              - {CONFIGS['max_tokens']}
    - Max mensagens histórico - {CONFIGS['max_messages']}
""")


CONFIGURAÇÕES
    - Modelo LLM              - llama-3.1-8b-instant
    - Temperatura             - 0.1
    - Max tokens              - 2048
    - Max mensagens histórico - 10



### FUNÇÕES AUXILIARES

In [32]:
# check conexão
def test_groq_connection() -> bool:
    """Testa a conexão com a API Groq."""
    try:
        llm = ChatGroq(
            model=CONFIGS['model'],
            temperature=0,
            api_key=GROQ_API_KEY
        )
        res = llm.invoke('Responda apenas: OK')
        if res.content.strip().upper() == 'OK':
            return True
        else:
            return False
    except Exception as e:
        print(f'Erro: {str(e)}')
        return False

### CLASSE PRINCIPAL

In [62]:
class ChatBot:
    """
    ChatBot para conversar através da API Groq
    """

    def __init__(self, configs:Dict[str,Any]=CONFIGS):
        """Inicializa o chatbot"""
        self.configs = configs
        self.llm = None
        self.history:List[Any] = []

        print('=== INICIALIZANDO CHATBOT ===')
        self._initialize_llm()
        self._initialize_system_prompt()

    def _initialize_llm(self):
        """Inicializa o modelo de linguagem Groq"""
        self.llm = ChatGroq(
            model=self.configs['model'],
            temperature=self.configs['temperature'],
            max_tokens=self.configs['max_tokens'],
            api_key=GROQ_API_KEY
        )
        print(f'Modelo {self.configs["model"]} carregado!')

    def _initialize_system_prompt(self):
        """Mensagem Fixa"""
        system_message = SystemMessage(
            content='Você é um assistente prestativo que responde perguntas objetivamente.'
        )
        self.history.append(system_message)

    def chat(self, user_input, verbose=True):
        """Processa um pergunta do usuário e retorna a resposta"""
        
        start_time = time.time()
        
        self.history.append(HumanMessage(content=user_input))
        res = self.llm.invoke(self.history)
        self.history.append(AIMessage(content=res.content))

        elapsed_time = time.time() - start_time

        if len(self.history)>self.configs['max_messages']:
            self.history = self.history[-self.configs['max_messages']:]
        
        if verbose:
            print(f'USER: {user_input}\n')
            print(f'BOT: {res.content}\nTime: {elapsed_time:2f}s')
            print(f'\n\nHistorico: {len(self.history)}')
        else:
            print(res.content)

### INICIALIZAR CHATBOT

In [71]:
if test_groq_connection():
    bot = ChatBot()

=== INICIALIZANDO CHATBOT ===
Modelo llama-3.1-8b-instant carregado!


### FAZER PERGUNTAS

In [72]:
bot.chat('Qual a capital da França')

USER: Qual a capital da França

BOT: A capital da França é Paris.
Time: 0.212864s


Historico: 3


In [73]:
bot.chat('Resuma econometria em uma frase.')

USER: Resuma econometria em uma frase.

BOT: A econometria é a aplicação de métodos estatísticos e matemáticos para analisar e prever fenômenos econômicos, permitindo a tomada de decisões informadas em áreas como política econômica, gestão de empresas e planejamento.
Time: 0.297423s


Historico: 5


In [74]:
bot.chat('Explique machine learning em forma de poema',verbose=False)

Em reinos de dados, onde informações se escondem,
Um novo rei surge, com poderes mágicos adornados.
Seu nome é Machine Learning, uma arte sublime,
Que permite ao computador aprender e se adaptar ao tempo.

Com algoritmos que se aprimoram, ele descobre e entende,
Os padrões escondidos, nos dados que se estendem.
Ele aprende a prever, a classificar e a escolher,
Com precisão cada vez maior, sem precisar de um mestre.

O treinamento é o processo, onde ele aprende a caminhar,
Com exemplos e feedback, ele se torna cada vez mais hábil.
O teste é o momento, onde ele demonstra sua força,
E o resultado é o que importa, para tomar decisões com segurança.

O Machine Learning é uma ferramenta poderosa,
Que pode ser usada em muitas áreas, sem limites ou fronteiras.
Ele pode ajudar a melhorar, a prever e a evitar,
Problemas e desafios, que antes pareciam insuperáveis.

Então, vamos explorar, os reinos do Machine Learning,
E descobrir suas possibilidades, sem limites ou medo de errar.
Porque com essa

In [75]:
bot.chat('resuma em um paragrafo: o que sao demonstrações financeiras na contabilidade?')

USER: resuma em um paragrafo: o que sao demonstrações financeiras na contabilidade?

BOT: As demonstrações financeiras são relatórios que apresentam a situação financeira e o desempenho de uma empresa em um determinado período. Elas são fundamentais para a tomada de decisões e para a avaliação da saúde financeira da empresa. As demonstrações financeiras incluem: o Balanço Patrimonial (que apresenta a situação patrimonial da empresa em um determinado momento), a Demonstração de Resultados do Exercício (que apresenta os resultados financeiros da empresa em um determinado período) e a Demonstração de Fluxo de Caixa (que apresenta a geração e o uso de caixa da empresa). Esses relatórios são preparados de acordo com as normas contábeis e são utilizados por investidores, credores e outros stakeholders para avaliar a viabilidade e a sustentabilidade da empresa.
Time: 0.781118s


Historico: 5


In [76]:
bot.chat('reescreva o poema')

USER: reescreva o poema

BOT: Em reinos de números, onde contas se escondem,
Um relatório surge, com informações adornados.
Seu nome é Demonstração, uma ferramenta sublime,
Que permite ao empresário entender e se adaptar ao tempo.

Com balanços e demonstrações, ele descobre e entende,
A situação financeira, dos negócios que se estendem.
Ele aprende a prever, a classificar e a escolher,
Com precisão cada vez maior, sem precisar de um mestre.

O balanço patrimonial é o processo, onde ele aprende a caminhar,
Com ativos e passivos, ele se torna cada vez mais hábil.
A demonstração de resultados é o momento, onde ele demonstra sua força,
E o resultado é o que importa, para tomar decisões com segurança.

As demonstrações financeiras são uma ferramenta poderosa,
Que pode ser usada em muitas áreas, sem limites ou fronteiras.
Elas podem ajudar a melhorar, a prever e a evitar,
Problemas e desafios, que antes pareciam insuperáveis.

Então, vamos explorar, os reinos das demonstrações,
E descobrir s

In [77]:
bot.chat('qual foi minha primeira pergunta?')

USER: qual foi minha primeira pergunta?

BOT: Sua primeira pergunta foi sobre o que são demonstrações financeiras na contabilidade.
Time: 0.262014s


Historico: 5
